# `Automated script to download Sentinel-1 images using Alaska Satellite Facility API`


# Import libraries and packages

In [17]:
!pip install asf_search
!pip install shapely
!pip install rasterio

In [18]:
import asf_search as asf
from shapely.geometry import box
from datetime import datetime, timezone
import os
import zipfile
import rasterio
from rasterio import shutil as rio_shutil
from google.colab import files
import getpass

# Authenticate with Earthdata NASA

In [19]:
# --- Prompt for Earthdata credentials securely ---
username = input("Enter your Earthdata username: ")
password = getpass.getpass("Enter your Earthdata password: ")

# --- Authenticate with Earthdata credentials ---
session = asf.ASFSession().auth_with_creds(username, password)

Enter your Earthdata username: carlosmendez1997
Enter your Earthdata password: ··········


# Define Area of Interest (AOI)

In [4]:
# --- Define bounding box coordinates ---
xmin, ymin, xmax, ymax = -68.803909, -11.033737, -68.725933, -10.979773

# --- Create shapely polygon and convert to WKT ---
bbox_geom = box(xmin, ymin, xmax, ymax)
bbox_wkt = bbox_geom.wkt

# Define search parameters

In [20]:
# --- Define search parameters ---
platform = "Sentinel-1A" # Satellite platform
polarizations = ["VV", "VH"] # Polarizations to include
beam_mode = "IW" # Beam mode
processing_level = "GRD" # Processing level
orbit_direction = "DESCENDING"
relative_orbit = 127 # Relative orbit number

# Set date range

In [21]:
# --- Date range ---
start_date = "2026-01-01" # Start date for search
end_date = datetime.now(timezone.utc).strftime("%Y-%m-%d") # Current date as end

# Perform Search of S1

In [22]:
# --- Perform search ---
results = asf.search(
    platform=platform,
    processingLevel=processing_level,
    beamMode=beam_mode,
    polarization=polarizations,
    relativeOrbit=relative_orbit,
    start=start_date,
    end=end_date,
    intersectsWith=bbox_wkt
)

# --- Print number of scenes found ---
print(f"Found {len(results)} Sentinel-1 scenes")

Found 16 Sentinel-1 scenes


# Create folder to dowload data

In [23]:
# --- Create folder for downloads ---
os.makedirs("ASF_Sentinel1", exist_ok=True)

# Download results

In [24]:
# --- Download results into folder ---
results.download(path="ASF_Sentinel1", session=session)

/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ASF_Sentinel1/S1_271782_IW1_20260207T101341_VV_84BC-BURST.tiff
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ASF_Sentinel1/S1_271782_IW1_20260207T101341_VH_84BC-BURST.tiff
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ASF_Sentinel1/OPERA_L2_RTC-S1_T127-271782-IW1_20260207T101341Z_20260207T222557Z_S1C_30_v1.0.h5
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping 

# Create folder only with Geotiff files

In [26]:
# --- Create output folder for GeoTIFFs ---
output_dir = "ASF_Sentinel1/GeoTIFFs"
os.makedirs(output_dir, exist_ok=True)

# --- Filter and copy only .tiff files (VV/VH) ---
for root, dirs, files in os.walk("ASF_Sentinel1"):
    for f in files:
        if f.endswith(".tiff") and ("VV" in f or "VH" in f):
            src_path = os.path.join(root, f)
            try:
                date_str = f.split("_")[3][:8]  # Extract date (YYYYMMDD)
            except Exception:
                date_str = "unknown"
            pol = "VV" if "VV" in f else "VH"
            out_name = f"S1_{pol}_{date_str}.tif"
            out_path = os.path.join(output_dir, out_name)

            # Copy GeoTIFF to output folder
            rio_shutil.copy(src_path, out_path, driver='GTiff')
            print(f"Saved {out_path}")

Saved ASF_Sentinel1/GeoTIFFs/S1_VV_20260126.tif
Saved ASF_Sentinel1/GeoTIFFs/S1_VH_20260108.tif
Saved ASF_Sentinel1/GeoTIFFs/S1_VV_20260102.tif
Saved ASF_Sentinel1/GeoTIFFs/S1_VH_20260207.tif
Saved ASF_Sentinel1/GeoTIFFs/S1_VH_20260126.tif
Saved ASF_Sentinel1/GeoTIFFs/S1_VH_20260102.tif
Saved ASF_Sentinel1/GeoTIFFs/S1_VV_20260108.tif
Saved ASF_Sentinel1/GeoTIFFs/S1_VV_20260207.tif


In [ ]:
# --- Compress all GeoTIFFs into a single ZIP ---
zip_path = "ASF_Sentinel1/Sentinel1_GeoTIFFs.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file)
        zipf.write(file_path, arcname=file)

print(f"✅ Compressed file ready: {zip_path}")

In [16]:
# --- Download ZIP to local machine ---
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>